Initialization

In [3]:
from telnetlib import STATUS
import psycopg2
import pandas as pd
import sys
from tqdm import tqdm

password = 'PAiac14-'
host = 'greenplum01.corral.tacc.utexas.edu'
port = 5432
database = 'uthealth'

username = 'piacobelli'
connection = psycopg2.connect(
    host = host, 
    database = database, 
    user = username, 
    port = port, 
    password = password, 
    keepalives = 1, keepalives_idle = 100
)
connection.autocommit = True

C:\Users\piacobelli\.conda\envs\piacobelliEnv\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\piacobelli\.conda\envs\piacobelliEnv\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\piacobelli\.conda\envs\piacobelliEnv\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


Row Counts

In [61]:
tableCreateQuery = '''
DROP TABLE IF EXISTS qa_reporting.dw_iqva_mbr_enrl_yearly_updated;
CREATE TABLE qa_reporting.dw_iqva_mbr_enrl_yearly_updated
(
    data_source TEXT,
    calendar_year INT,
    table_src TEXT,
    dw_row_count INT,
    dw_uth_mbr_id_count INT,
    dw_src_mbr_id_count INT,
    src_mbr_count INT,
    mbr_count_diff INT,
    mbr_count_percentage FLOAT,
    dw_mm INT,
    src_mm INT,
    mm_diff INT,
    mm_diff_percentage FLOAT,
    date_generated DATE
);
'''

with connection.cursor() as cursor:
    cursor.execute(tableCreateQuery)

with connection.cursor() as cursor:
      tableInsertQuery = '''
            INSERT INTO qa_reporting.dw_iqva_mbr_enrl_yearly_updated
            (data_source, calendar_year, table_src, dw_row_count, dw_uth_mbr_id_count, dw_src_mbr_id_count, dw_mm,date_generated)
            SELECT data_source, 
                  year, 
                  table_id_src, 
                  count(*),
                  count(distinct uth_member_id),
                  count(distinct member_id_src),
                  sum(total_enrolled_months),
                  now()::date
            FROM dw_staging.iqva_member_enrollment_yearly
            GROUP BY 1,2,3;
      '''

      cursor.execute(tableInsertQuery)


with connection.cursor() as cursor:
      tableUpdateMbrQuery = '''
            UPDATE qa_reporting.dw_iqva_mbr_enrl_yearly_updated a
            SET src_mbr_count = b.pat_count,
                  mbr_count_diff = a.dw_src_mbr_id_count - b.pat_count,
                  mbr_count_percentage = 100. * abs(a.dw_src_mbr_id_count - b.pat_count) / b.pat_count
            FROM qa_reporting.iqvia_counts b
            WHERE data_source = 'iqva'
            AND a.calendar_year = b.year
            AND a.table_src ||'2' = b.table_name;
      '''

      cursor.execute(tableUpdateMbrQuery)

      enrlMonthsQuery = '''
            WITH enrl_months AS (
                  SELECT DISTINCT year, pat_id, month_id
                  FROM iqvia.enroll2
            ),
            enrl_my AS (
                  SELECT year, COUNT(*) as mm
                  FROM enrl_months
                  GROUP BY 1
            )
            UPDATE qa_reporting.dw_iqva_mbr_enrl_yearly_updated a
            SET src_mm = b.mm, mm_diff = a.dw_mm - b.mm, mm_diff_percentage = abs(a.dw_mm - b.mm) / b.mm
            FROM enrl_my b
            WHERE data_source = 'iqva' AND a.calendar_year = b.year;
      '''

      cursor.execute(enrlMonthsQuery)

countsDf = pd.read_sql('''SELECT * 
FROM qa_reporting.dw_iqva_mbr_enrl_yearly_updated 
WHERE table_src = 'enroll'
ORDER BY mm_diff ;''', 
con=connection)

countsDf

C:\Users\piacobelli\AppData\Local\Temp\6\ipykernel_37504\2814561387.py:77: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  countsDf = pd.read_sql('''SELECT *


,data_source,calendar_year,table_src,dw_row_count,dw_uth_mbr_id_count,dw_src_mbr_id_count,src_mbr_count,mbr_count_diff,mbr_count_percentage,dw_mm,src_mm,mm_diff,mm_diff_percentage,date_generated
0,iqva,2022,enroll,10691149,10691149,10691149,10691149,0,0.0,99941204,99941204,0,0.0,2024-04-07
1,iqva,2023,enroll,7759744,7759744,7759744,7759744,0,0.0,23662763,23662763,0,0.0,2024-04-07
2,iqva,2019,enroll,13625173,13625173,13625173,13625173,0,0.0,128691131,128691131,0,0.0,2024-04-07
3,iqva,2007,enroll,34820782,34820782,34820782,34820782,0,0.0,346468836,346468836,0,0.0,2024-04-07
4,iqva,2010,enroll,31386216,31386216,31386216,31386216,0,0.0,312564976,312564976,0,0.0,2024-04-07
5,iqva,2014,enroll,28543784,28543784,28543784,28543784,0,0.0,258722696,258722696,0,0.0,2024-04-07
6,iqva,2008,enroll,37240842,37240842,37240842,37240842,0,0.0,373065024,373065024,0,0.0,2024-04-07
7,iqva,2006,enroll,30675914,30675914,30675914,30675914,0,0.0,284739058,284739058,0,0.0,2024-04-07
8,iqva,2018,enroll,13520651,13520651,13520651,13520651,0,0.0,131127502,131127502,0,0.0,2024-04-07
9,iqva,2020,enroll,11947614,11947614,11947614,11947614,0,0.0,108287386,108287386,0,0.0,2024-04-07


Gender

In [33]:
genderQuery = '''
    WITH iqva_gen_cd as (
        SELECT DISTINCT year, a.pat_id,
        CASE 
            WHEN b.pat_id IS NULL THEN 'U'
            ELSE b.der_sex
        END AS der_sex
    FROM iqvia.enroll2 a
    LEFT JOIN iqvia.enroll_synth b ON a.pat_id = b.pat_id
    ),
    iqva_gen AS (
        SELECT year, der_sex, COUNT(*) gender_count
        FROM iqva_gen_cd
        GROUP BY 1,2
    ), dw_gen AS (
        SELECT year, gender_cd, COUNT(*) gender_count
        FROM dw_staging.iqva_member_enrollment_yearly
        WHERE claim_created_flag IS NOT true
        GROUP BY 1,2
    )
    SELECT a.*, b.gender_count AS src_gender_count, 
            a.gender_count - b.gender_count AS gender_diff, 
            100. * abs(a.gender_count - b.gender_count) / b.gender_count AS gender_diff_percent
    FROM iqva_gen b
    JOIN dw_gen a
    ON a.year = b.year
    AND a.gender_cd = b.der_sex
    ORDER BY year, gender_cd;
'''

pd.read_sql(genderQuery,  con=connection)

C:\Users\piacobelli\AppData\Local\Temp\6\ipykernel_54556\4030774489.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(genderQuery,  con=connection)


,year,gender_cd,gender_count,src_gender_count,gender_diff,gender_diff_percent
0,2006,F,1154719,1154719,0,0.0
1,2006,M,1053423,1053423,0,0.0
2,2006,U,28467772,28467772,0,0.0
3,2007,F,1214513,1214513,0,0.0
4,2007,M,1110803,1110803,0,0.0
5,2007,U,32495466,32495466,0,0.0
6,2008,F,1271276,1271276,0,0.0
7,2008,M,1166103,1166103,0,0.0
8,2008,U,34803463,34803463,0,0.0
9,2009,F,1299655,1299655,0,0.0


Plan Type

In [30]:
planTypeQuery = '''
    WITH iqva_enroll AS (
        SELECT year, pat_id, plan_type, month_id
        FROM iqvia.enroll2 a
        LEFT JOIN reference_tables.ref_plan_type c ON c.data_source  = 'iqva' AND c.plan_type_src = a.prd_type
    ),
    iqva_agg_enroll AS (
        SELECT year, pat_id, plan_type, MAX(month_id) my, COUNT(*)
        FROM iqva_enroll
        GROUP BY 1,2,3
    ),
    iqva_grp_enroll AS (
        SELECT *, row_number() OVER(PARTITION BY pat_id, year ORDER BY COUNT DESC, my desc) AS my_grp
        FROM iqva_agg_enroll
    ),
    iqva_plans AS (
        SELECT year, plan_type, COUNT(*) plan_count
        FROM iqva_grp_enroll 
        WHERE my_grp = 1
        GROUP BY 1,2
    ),
    dw_plans AS (
        SELECT year, plan_type, COUNT(*) plan_count
        FROM dw_staging.iqva_member_enrollment_yearly
        WHERE claim_created_flag IS NOT true
        GROUP BY 1,2
    )
    SELECT a.*, b.plan_count AS src_plan_count, 
            a.plan_count - b.plan_count AS plan_diff, 
            100. * abs(a.plan_count - b.plan_count) / b.plan_count AS plan_diff_percent
    FROM iqva_plans b
    JOIN dw_plans a ON a.year = b.year AND a.plan_type = b.plan_type
    ORDER BY year;
'''

planTypeCountsDf = pd.read_sql(planTypeQuery,  con=connection)
planTypeCountsDf

C:\Users\piacobelli\AppData\Local\Temp\6\ipykernel_54556\1031229794.py:36: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  planTypeCountsDf = pd.read_sql(planTypeQuery,  con=connection)


,year,plan_type,plan_count,src_plan_count,plan_diff,plan_diff_percent
0,2006,HMO,8494010,8493640,370,0.004356
1,2006,POS,2450542,2450526,16,0.000653
2,2006,UNK,230742,230742,0,0.000000
3,2006,CDHP,96372,96372,0,0.000000
4,2006,PPO,17974982,17974966,16,0.000089
...,...,...,...,...,...,...
101,2022,UNK,285607,285607,0,0.000000
102,2023,PPO,4196810,4196810,0,0.000000
103,2023,UNK,196659,196659,0,0.000000
104,2023,CDHP,1195719,1195719,0,0.000000


In [66]:
planTypeCountsDf.groupby('year')['plan_diff_percent'].min(), planTypeCountsDf.groupby('year')['plan_diff_percent'].max()

(year
 2006    0.0
 2007    0.0
 2008    0.0
 2009    0.0
 2010    0.0
 2011    0.0
 2012    0.0
 2013    0.0
 2014    0.0
 2015    0.0
 2016    0.0
 2017    0.0
 2018    0.0
 2019    0.0
 2020    0.0
 2021    0.0
 2022    0.0
 2023    0.0
 Name: plan_diff_percent, dtype: float64,
 year
 2006    0.004356
 2007    0.025240
 2008    0.009730
 2009    0.004635
 2010    0.005091
 2011    0.006395
 2012    0.008457
 2013    0.020564
 2014    0.016726
 2015    0.097561
 2016    0.329489
 2017    0.007529
 2018    0.000000
 2019    0.000000
 2020    0.000000
 2021    0.000000
 2022    0.000000
 2023    0.000000
 Name: plan_diff_percent, dtype: float64)

In [67]:
planTypeCountsDf.sort_values('plan_diff_percent')

,year,plan_type,plan_count,src_plan_count,plan_diff,plan_diff_percent
105,2023,HMO,2170556,2170556,0,0.000000
91,2020,PPO,6729247,6729247,0,0.000000
92,2021,HMO,3046863,3046863,0,0.000000
57,2015,UNK,282138,282138,0,0.000000
56,2014,CDHP,1655661,1655661,0,0.000000
...,...,...,...,...,...,...
10,2007,HMO,8472771,8470633,2138,0.025240
67,2016,HMO,10412895,10410198,2697,0.025907
68,2016,POS,963062,962757,305,0.031680
60,2015,FFS,1026,1025,1,0.097561


In [68]:
plantypesAggDf = planTypeCountsDf.groupby('year')[['plan_count', 'src_plan_count', 'plan_diff']].sum()
plantypesAggDf['percent_diff'] = 100. * plantypesAggDf['plan_diff'] / plantypesAggDf['src_plan_count'] 
plantypesAggDf

,plan_count,src_plan_count,plan_diff,percent_diff
year,,,,
2006,30673576,30673174,402,0.001311
2007,34805575,34803305,2270,0.006522
2008,37224619,37223637,982,0.002638
2009,35200915,35200456,459,0.001304
2010,31377523,31377138,385,0.001227
2011,31072095,31071435,660,0.002124
2012,28202333,28201628,705,0.002500
2013,25556325,25554490,1835,0.007181
2014,28508226,28506519,1707,0.005988


State

In [25]:
stateQuery= '''
    WITH iqva_enroll as (
        SELECT DISTINCT year, a.pat_id, COALESCE(pat_state, 'XX') pat_state
        FROM iqvia.enroll2 a
        LEFT JOIN iqvia.enroll_synth b ON a.pat_id = b.pat_id
    ),
    iqva_states as (
        SELECT year, pat_state, COUNT(*) state_count
        FROM iqva_enroll m
        GROUP BY 1,2
    ), 
    dw_plans AS (
        SELECT year, CASE WHEN state IS NULL THEN 'XX' ELSE state END AS state, COUNT(*) state_count
        FROM dw_staging.iqva_member_enrollment_yearly
        WHERE claim_created_flag IS NOT true
        GROUP BY 1,2
    )
    SELECT a.*, b.state_count as src_state_count, 
            a.state_count - b.state_count as plan_diff, 
            100. * abs(a.state_count - b.state_count) / b.state_count as plan_diff_percent
    FROM iqva_states b
    FULL JOIN dw_plans a
    ON a.year = b.year
    AND a.state = b.pat_state
    ORDER BY year;
'''

stateDf = pd.read_sql(stateQuery,  con=connection)
stateDf

C:\Users\piacobelli\AppData\Local\Temp\6\ipykernel_54556\808172135.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  stateDf = pd.read_sql(stateQuery,  con=connection)


,year,state,state_count,src_state_count,plan_diff,plan_diff_percent
0,2006,KS,1109,1109,0,0.0
1,2006,SD,443,443,0,0.0
2,2006,VA,292377,292377,0,0.0
3,2006,PA,5215,5215,0,0.0
4,2006,AK,177,177,0,0.0
...,...,...,...,...,...,...
931,2023,CO,283280,283280,0,0.0
932,2023,OR,347546,347546,0,0.0
933,2023,XX,51945,51945,0,0.0
934,2023,CT,223802,223802,0,0.0


In [26]:
stateDf.sort_values('plan_diff_percent', ascending=False)

,year,state,state_count,src_state_count,plan_diff,plan_diff_percent
0,2006,KS,1109,1109,0,0.0
628,2018,SC,21698,21698,0,0.0
616,2017,IA,37966,37966,0,0.0
617,2017,CO,172738,172738,0,0.0
618,2017,AK,781,781,0,0.0
...,...,...,...,...,...,...
316,2012,KY,214459,214459,0,0.0
317,2012,MS,1746,1746,0,0.0
318,2012,GA,161277,161277,0,0.0
319,2012,TX,26651,26651,0,0.0


In [27]:
stateDf['state'].unique()

array(['KS', 'SD', 'VA', 'PA', 'AK', 'NY', 'ND', 'RI', 'ME', 'NM', 'CO',
       'FL', 'XX', 'CA', 'ID', 'GA', 'VT', 'IL', 'NE', 'UT', 'HI', 'TN',
       'DC', 'OK', 'AR', 'AL', 'MA', 'MN', 'DE', 'TX', 'IA', 'NV', 'OR',
       'NC', 'WI', 'OH', 'WA', 'MD', 'WY', 'AZ', 'MI', 'NJ', 'MT', 'IN',
       'WV', 'NH', 'MO', 'SC', 'KY', 'LA', 'CT', 'MS'], dtype=object)

In [73]:
stateDf['state'].unique().shape

(52,)

In [28]:
stateDf[stateDf['state'] == '']

,year,state,state_count,src_state_count,plan_diff,plan_diff_percent


In [75]:
stateDf[stateDf['state'].isna()]

,year,state,state_count,src_state_count,plan_diff,plan_diff_percent
